# Practica 2

In [7]:
import nltk
import numpy as np
from nltk.stem.snowball import SnowballStemmer
#nltk.download()

In [2]:
with open('corpusML.txt', 'r') as f:
    corpus = f.readlines()

In [5]:
corpus[0:2]

['Comencé a trabajar y me pegaron, me maltrataron con chicote \n',
 'Mis patrones me pegaron porque no me quería apurar, porque era flojo \n']

### 1, 2) Limpiar corpus y agregar simbolos de inicio y fin

In [49]:
stemmer = SnowballStemmer("spanish")
stems = []
corpus_processed = []

for sentence in corpus:
    tokens = nltk.word_tokenize(sentence)               # Obtener lista tokens
    for tk in tokens:   
        if tk.isalpha():                                # Validar token como caracter del alfabeto                                
            stem = stemmer.stem(tk)                     # Aplicar algotimo de stemming
            stems.append(stem)                 
    s = '<BOS> ' + ' '.join(stems) + ' <EOS>'           # Agregar simbolos de inicio y fin
    stems.clear()
    corpus_processed.append(s)

In [52]:
corpus_processed[0:2]

['<BOS> comenc a trabaj y me peg me maltrat con chicot <EOS>',
 '<BOS> mis patron me peg porqu no me quer apur porqu era floj <EOS>']

### 3) Obtener los bigramas

In [71]:
# Funcion para obtener los bigramas de una secuencia de caracteres
def bigrams(sequence):
    s = sequence.split()
    return [(wi, wj) for wi, wj in zip(s[:-1], s[1:])]

In [77]:
bigrams(corpus_processed[0])

[('<BOS>', 'comenc'),
 ('comenc', 'a'),
 ('a', 'trabaj'),
 ('trabaj', 'y'),
 ('y', 'me'),
 ('me', 'peg'),
 ('peg', 'me'),
 ('me', 'maltrat'),
 ('maltrat', 'con'),
 ('con', 'chicot'),
 ('chicot', '<EOS>')]

AttributeError: 'list' object has no attribute 'split'